# <div align="center" style="color: #ff5733;">Data Requested By Annastasia for Monthly Reporting</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
# Year and Month for which data is extracted
data_year = 2024
data_month = 6

In [3]:
# How many Applications applied to each month

sq = """
# loan applied
SELECT 
    DATE_TRUNC( startApplyDateTime,MONTH) as mm, 
    case when reloan_flag = 1  and loantype not like 'FLEXUP' then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
         
         else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as StartedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 

group by 1,2
order by 1 desc,2;
"""

StartedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in StartedApps are:\t {StartedApps.shape}")

Job ID e3c7b401-6399-4e3c-a130-7759149d937c successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in StartedApps are:	 (186, 3)


In [4]:
# How many submitted applications to each month
sq = """
# loan Submitted
SELECT 
    DATE_TRUNC((case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then startApplyDateTime else termsAndConditionsSubmitDateTime end),MONTH) as mm,  
    case when reloan_flag = 1 and loantype not like 'FLEXUP'then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as SubmittedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
group by 1,2
order by 1 desc,2;
"""

SubmittedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns of SubmittedApps are:\t {SubmittedApps.shape}")
SubmittedApps.sample(10)

Job ID 9856d549-c39e-4922-b825-62a3725d3a36 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns of SubmittedApps are:	 (187, 3)


,mm,LoanProduct,SubmittedApps
155,2022-03-01,SIL-Online(discontinued),394
50,2023-11-01,Reloan,181
33,2024-02-01,SIL-Instore,7082
103,2023-02-01,Quick,14371
176,2021-08-01,SIL-Online(discontinued),1
174,2021-08-01,Flex,51
6,2024-06-01,Flex-up,221
111,2022-12-01,Big Loan,11
95,2023-03-01,Flex,6047
106,2023-01-01,Big Loan,6


In [5]:
# Approved Loans for each month

sq = """
# Approved loans
SELECT 
DATE_TRUNC((case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then startApplyDateTime 
                  when reloan_flag = 1 and loantype not like 'FLEXUP' then startApplyDateTime else decision_date end),MONTH) as mm,
        case when reloan_flag = 1 and loantype not like 'FLEXUP'then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as ApprovedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
where 
(case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then flagDisbursement 
      when reloan_flag = 1 and loantype not like 'FLEXUP' then flagDisbursement  else flagApproval end) = 1
group by 1,2
order by 1 desc,2
;
"""

ApprovedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in ApprovedApps are:\t {ApprovedApps.shape}")
ApprovedApps.head(10)

Job ID 6bef59a7-94f4-4d2d-8eec-4cb9c54c29d1 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in ApprovedApps are:	 (154, 3)


,mm,LoanProduct,ApprovedApps
0,2024-07-01,Flex-up,7
1,2024-07-01,Quick,37
2,2024-07-01,Reloan,15
3,2024-07-01,SIL-Instore,364
4,2024-06-01,ACL TSA,3
5,2024-06-01,Flex-up,221
6,2024-06-01,Quick,1568
7,2024-06-01,Reloan,226
8,2024-06-01,SIL-Instore,12060
9,2024-05-01,ACL TSA,2


In [6]:
# Booked Loans for each month

sq = """
#Booked loans
SELECT 
DATE_TRUNC( disbursementDateTime,MONTH) as mm, 
            case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as BookedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
    where flagDisbursement = 1

group by 1,2
order by 1 desc,2
;
"""

BookedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in BookedApps are:\t {BookedApps.shape}")
BookedApps.head(10)

Job ID b15f78d6-14e9-4916-8da2-4d475b93e702 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in BookedApps are:	 (153, 3)


,mm,LoanProduct,BookedApps
0,2024-07-01,Flex-up,12
1,2024-07-01,Quick,32
2,2024-07-01,Reloan,15
3,2024-07-01,SIL-Instore,360
4,2024-06-01,ACL TSA,3
5,2024-06-01,Flex-up,351
6,2024-06-01,Quick,1544
7,2024-06-01,Reloan,226
8,2024-06-01,SIL-Instore,11966
9,2024-05-01,ACL TSA,2


In [7]:
# Get the total booked amount for a month

sq = """
# Booked Amount
SELECT 
DATE_TRUNC( disbursementDateTime,MONTH) as mm, 
    case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    sum (disbursedLoanAmount) as BookedAmt
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
     where flagDisbursement = 1
group by 1,2
order by 1 desc,2
;
"""
BookedAmt = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in BookedAmt are:\t {BookedAmt.shape}")
BookedAmt.head(10)

Job ID 859a68d5-2db2-440a-bdb2-102ff2339a6c successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in BookedAmt are:	 (153, 3)


,mm,LoanProduct,BookedAmt
0,2024-07-01,Flex-up,7.252971e+05
1,2024-07-01,Quick,4.545000e+05
2,2024-07-01,Reloan,2.768826e+05
3,2024-07-01,SIL-Instore,3.500134e+06
4,2024-06-01,ACL TSA,3.500000e+04
5,2024-06-01,Flex-up,1.422276e+07
6,2024-06-01,Quick,1.985180e+07
7,2024-06-01,Reloan,5.904780e+06
8,2024-06-01,SIL-Instore,1.187000e+08
9,2024-05-01,ACL TSA,4.000000e+04


In [8]:
BookedAmt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   mm           153 non-null    datetime64[us]
 1   LoanProduct  153 non-null    object        
 2   BookedAmt    153 non-null    float64       
dtypes: datetime64[us](1), float64(1), object(1)
memory usage: 3.7+ KB


In [9]:
raw_data = StartedApps.merge(SubmittedApps,how='outer').merge(ApprovedApps,how='outer').merge(BookedApps,how='outer').merge(BookedAmt,how='outer')

In [10]:
raw_data.tail(10)

,mm,LoanProduct,StartedApps,SubmittedApps,ApprovedApps,BookedApps,BookedAmt
179,2024-06-01,ACL TSA,14,10,3,3,3.500000e+04
180,2024-06-01,Flex,30,10,<NA>,<NA>,NaN
181,2024-06-01,Flex-up,230,221,221,351,1.422276e+07
182,2024-06-01,Quick,54482,40355,1568,1544,1.985180e+07
183,2024-06-01,Reloan,243,234,226,226,5.904780e+06
184,2024-06-01,SIL-Instore,20390,20183,12060,11966,1.187000e+08
185,2024-07-01,Flex-up,7,7,7,12,7.252971e+05
186,2024-07-01,Quick,1837,1430,37,32,4.545000e+05
187,2024-07-01,Reloan,17,15,15,15,2.768826e+05
188,2024-07-01,SIL-Instore,671,662,364,360,3.500134e+06


In [11]:
print(f'{data_year}-{data_month:02d}-01')

2024-06-01


In [12]:
raw_data_current_month = raw_data[(raw_data['mm']>='2022-08-01') &(raw_data['mm']<=f'{data_year}-{data_month:02d}-01')]

In [13]:
raw_data_current_month

,mm,LoanProduct,StartedApps,SubmittedApps,ApprovedApps,BookedApps,BookedAmt
53,2022-08-01,Flex,5558,4732,594,599,1.198500e+07
54,2022-08-01,Quick,43764,30660,1910,1926,3.300100e+07
55,2022-08-01,SIL-Instore,967,956,456,457,1.137941e+07
56,2022-08-01,SIL-Online(discontinued),1192,652,41,41,1.197499e+06
57,2022-09-01,Big Loan,13,12,<NA>,<NA>,NaN
...,...,...,...,...,...,...,...
180,2024-06-01,Flex,30,10,<NA>,<NA>,NaN
181,2024-06-01,Flex-up,230,221,221,351,1.422276e+07
182,2024-06-01,Quick,54482,40355,1568,1544,1.985180e+07
183,2024-06-01,Reloan,243,234,226,226,5.904780e+06


### Loan Account Tagging

In [14]:
sq="""
# Loan Tagging
SELECT
    loanAccountNumber,
    Case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    disbursementDateTime,
    (CASE WHEN new_loan_type = 'Flex-up' then
    LAG(new_loan_type) OVER (PARTITION BY customerId ORDER BY disbursementDateTime) END) AS OriginalLoanProduct
  FROM
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
   where flagDisbursement=1
;
"""
loan_type = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in loan_type are: {loan_type.shape}")
loan_type.tail(10)

Job ID 2f01bfbe-9a09-47a5-9d5d-5ee4bb69ea50 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in loan_type are: (152898, 4)


,loanAccountNumber,LoanProduct,disbursementDateTime,OriginalLoanProduct
152888,60826220900018,SIL-Instore,2024-06-30 13:50:57,None
152889,60826222310013,SIL-Instore,2024-06-30 14:18:38,None
152890,60826222450019,SIL-Instore,2024-06-30 14:31:53,None
152891,60826227510012,SIL-Instore,2024-06-30 16:32:31,None
152892,60826228240012,SIL-Instore,2024-06-30 16:42:28,None
152893,60826229510018,SIL-Instore,2024-06-30 17:07:07,None
152894,60826232180011,SIL-Instore,2024-06-30 18:23:32,None
152895,60826248550017,SIL-Instore,2024-07-01 12:56:32,None
152896,60826252840018,SIL-Instore,2024-07-01 16:03:30,None
152897,60826254740012,SIL-Instore,2024-07-01 15:58:13,None


In [15]:
Loan_Acs_current_month = loan_type[(loan_type['disbursementDateTime'].dt.month == data_month) & (loan_type['disbursementDateTime'].dt.year == data_year)]

In [16]:
Loan_Acs_current_month = Loan_Acs_current_month[['loanAccountNumber', 'LoanProduct','OriginalLoanProduct']]

#### VAS

In [17]:
sq = """
# Vas 
SELECT 
    extract(year from disbursementDateTime) as year , 
    extract(month from disbursementDateTime) as month,
    Case when reloan_flag = 1 and b.loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when b.loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as new_loan_type, 
    count(distinct  b.loanAccountNumber ) AS TotalSold,
   
   count(distinct CASE WHEN vas_flag = 'true' THEN b.loanAccountNumber END) AS SoldWithVAS
     FROM `prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application` a join
`prj-prod-dataplatform.risk_credit_mis.loan_master_table` b on a.loanAccountNumber = b.loanAccountNumber
where flagDisbursement = 1
group by 1,2 ,3 
order by 1 desc,2 desc,3
;
"""

vas_df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in vas_df are:\t {vas_df.shape}")
vas_df.head(10)

Job ID fa6cfd7c-28f4-44b4-9ff5-02cb3e96fcf5 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in vas_df are:	 (153, 5)


,year,month,new_loan_type,TotalSold,SoldWithVAS
0,2024,7,Flex-up,12,0
1,2024,7,Quick,32,22
2,2024,7,Reloan,15,0
3,2024,7,SIL-Instore,360,228
4,2024,6,ACL TSA,3,2
5,2024,6,Flex-up,351,0
6,2024,6,Quick,1544,910
7,2024,6,Reloan,226,0
8,2024,6,SIL-Instore,11966,8224
9,2024,5,ACL TSA,2,2


In [18]:
vas_df_current_month = vas_df[(vas_df['year']==data_year)& (vas_df['month']==data_month)]

In [19]:
with pd.ExcelWriter(f'loan_accounts_tagging_{data_year}_{data_month:02d}_v2.xlsx') as writer:
    # Write each DataFrame to a specific sheet in the Excel file
    raw_data_current_month.to_excel(writer, sheet_name='raw_data', index=False)
    Loan_Acs_current_month.to_excel(writer, sheet_name='Loan Acs', index=False)
    vas_df_current_month.to_excel(writer, sheet_name='VAS', index=False)